Generating the artificial dataset for points in the circle with the guassian noise
<uo>
    <li>Each training example consists of four features</li>
    <li>Dataset consists of 1000 training example and 400 testing example</li>
<uo>
    

In [3]:
# Generate a random points in the circle (with uniform distribution) given the x, y cordinates 
# of the center of circle and radius
import random 
import math
import numpy as np
from numpy.random import normal
class pointInCircle:
    def __init__(self):
        self.r =random.uniform(0,101)
        self.x =random.uniform(-50,50)
        self.y =random.uniform(-50,50)
            
    def randPoint(self):  
        '''generates center point and radius for the circle with uniform distribution in some range'''
        self.r =random.uniform(0,101)
        self.x =random.uniform(-50,50)
        self.y =random.uniform(-50,50)
        
        #genearate three point in the circle with uniform distribution
        points1 = []
        for i in range(0,3):
            #random theta
            theta = random.uniform(0, 2*math.pi)
            
            # point in the circle given center (x,y)
            #math logic: ((x+ rcos(theta)-x)^2 +(y+ rcos(theta)-y)^2 = r^2)
            point = (self.x + self.r*math.cos(theta), self.y + self.r*math.sin(theta))
            points1 += point                 
        return(points1)
    
    def guassianDistPoint(self):
        points2 = normal(loc=0, scale=0.01, size=6) #this is numpy array of 1X6
        return(points2.tolist()) #converting numpy array to list
        
    
    def pointsForTraining(self):
        ''' three points in the circle as list in order x1, y1, x2, y2, x3, y3, x_center, y_center
        with added guassian noise to points in circle only'''
        sum_list = [a + b for a, b in zip(self.randPoint(),self.guassianDistPoint())]
        sum_list.append(self.x)
        sum_list.append(self.y)
        #print(self.r)
        return(sum_list)
    

In [4]:
h = pointInCircle()
#h.randPoint() 
#h.guassianDistPoint()
#h.pointsForTraining()

In [5]:
import pandas as pd

train_data_list =[]
for i in range (0,1000):
    train_data_list.append(h.pointsForTraining())
    
test_data_list =[]
for j in range(0,500):
    test_data_list.append(h.pointsForTraining())
    
circles_train_df =pd.DataFrame(train_data_list, columns =['x1','y1','x2','y2','x3','y3','h','k'])
circles_test_df =pd.DataFrame(test_data_list, columns =['x1','y1','x2','y2','x3','y3','h','k'])

In [6]:
#circles_train_df

In [7]:
#circles_test_df

In [8]:
#Convert the pandas dataframe file to csv file
circles_train_df.to_csv('circle_trainpoints.csv', index=False)
circles_test_df.to_csv('circle_testpoints.csv', index=False)

Preparing the data
-seperating the features and labels in the training and testing data set

In [9]:
import torch
#print(torch.__version__)

In [10]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self,trainpoints, testpoints):
        circle_train_df = pd.read_csv(trainpoints)
        circle_test_df = pd.read_csv(testpoints)
        
        self.circle_train_df = circle_train_df.iloc[:,:].values
        self.circle_test_df = circle_test_df.iloc[:,:].values
        
        self.x_circle_train_df = circle_train_df.iloc[:,0:6].values
        self.y_circle_train_df = circle_train_df.iloc[:,6:7].values
        
        self.x_circle_test_df = circle_test_df.iloc[:,0:6].values        
        self.y_circle_test_df = circle_test_df.iloc[:,6:7].values
        
        self.x_tensor_circle_train_df = torch.tensor(self.x_circle_train_df,dtype=torch.float64) 
        self.y_tensor_circle_train_df = torch.tensor(self.y_circle_train_df,dtype=torch.float64) 
        
        self.x_tensor_circle_test_df = torch.tensor(self.x_circle_test_df,dtype=torch.float64) 
        self.y_tensor_circle_test_df = torch.tensor(self.y_circle_test_df,dtype=torch.float64) 
                
           
    def __len__(self):
        return len(self.x_tensor_circle_train_df),len(self.y_tensor_circle_train_df), len(self.x_tensor_circle_test_df), len(self.y_tensor_circle_test_df)
  
    def __getitem__(self,idx):
        return self.x_tensor_circle_train_df[idx],self.y_tensor_circle_train_df[idx]

In [11]:
p=MyDataset('circle_trainpoints.csv','circle_testpoints.csv')

In [12]:
#p.circle_train_df

In [13]:
#p.__getitem__(2)

In [14]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

read and print the first small batch of data examples. The shape of the features in each minibatch tells us both the minibatch size and the number of input features. Likewise, our minibatch of labels will have a shape given by batch_size.

As we run the iteration, we obtain distinct minibatches successively until the entire dataset has been exhausted 

In [15]:
batch_size = 10
features = p.x_circle_train_df
#features=  torch.from_numpy(features)
labels = p.y_circle_train_df 
#labels = torch.from_numpy(labels)

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

[[-2.66320204e+01 -1.31991733e+01 -2.76935242e+01 -1.39482103e+01
  -4.04483163e+01 -4.92888819e+01]
 [-4.19819915e+01  5.61925752e+00  5.99621019e+01 -2.36554231e+01
   6.70389760e+01  1.84318922e+01]
 [-8.78005597e+01 -4.43963244e+01 -9.22416901e+01  8.29880817e+01
   5.72432120e+01 -1.45319881e+01]
 [-6.81805651e-01 -7.29242259e+01 -3.94005196e+01  2.60980653e+00
  -8.60439183e+01 -7.48370784e+01]
 [ 2.42479489e+01  1.21621218e+02  1.98952331e+01 -2.40281540e+01
  -5.04331580e+01  6.77635538e+01]
 [ 5.21136063e+01 -2.34756065e+01  7.21089169e+01  9.98132866e-02
   2.81837766e+01 -3.57382264e+01]
 [-3.30643081e+01 -3.03018071e+01 -2.25882484e+01  5.05196403e+00
  -2.34618914e+01  5.17536117e+00]
 [ 1.26786938e+01 -7.89841854e+00 -3.43590848e+01  7.76143206e+00
  -2.01006872e+01 -1.16129460e+01]
 [ 9.43365163e+01  3.24274676e+01  2.06208738e+01 -8.10011106e+01
   7.20746576e+01  5.87424995e+01]
 [-2.49601710e+01  3.73365946e+01 -8.50363170e+01  2.81741926e+01
   1.95555120e+01 -7.4941

In [ ]:
#Initializing the model parameter

initializing weights by sampling random numbers from a normal distribution with mean 0 and a standard deviation of 0.01, and setting the bias to 0.

In [ ]:
w = torch.normal(0, 0.01, size=(6,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

updating them until they fit our data sufficiently well. Each update requires taking the gradient of our loss function with respect to the parameters. Given this gradient, we can update each parameter in the direction that may reduce the loss.


In [ ]:
def linreg(X, w, b):  #@save
    return torch.matmul(X, w) + b

Defining the loss function

In [ ]:
def squared_loss(y_hat, y):  
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

Optimizing Algorithm

In [11]:
def sgd(params, lr, batch_size):  
    '''Minibatch stochastic gradient descent.'''
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [ ]:
#Training
#automatic differentiation, to compute the gradient.
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')